 # Download Module

In [3]:
!pip install tensorflow

# Importing the Modules

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Importing the MNIST Keras Model and Assigning the data

In [5]:
(X_train , y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

# Data Preprocessing

In [6]:
X_train = X_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
X_test = X_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# Creating the CNN model

In [7]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)))
cnn.add(tf.keras.layers.MaxPooling2D(2,2))
cnn.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
cnn.add(tf.keras.layers.MaxPooling2D(2,2))
cnn.add(tf.keras.layers.Conv2D(64, (3,3), activation='relu'))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(64, activation='relu'))
cnn.add(tf.keras.layers.Dense(10, activation='softmax'))

cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\Brian Biju\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


# Training the Model

In [8]:
cnn.fit(X_train, y_train , epochs=5, batch_size=64, validation_split=0.1)

cnn.save('model.h5')

Epoch 1/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 8s 8ms/step - accuracy: 0.8385 - loss: 0.4977 - val_accuracy: 0.9735 - val_loss: 0.0846
Epoch 2/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9813 - loss: 0.0587 - val_accuracy: 0.9860 - val_loss: 0.0475
Epoch 3/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9869 - loss: 0.0427 - val_accuracy: 0.9830 - val_loss: 0.0565
Epoch 4/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9899 - loss: 0.0302 - val_accuracy: 0.9887 - val_loss: 0.0418
Epoch 5/5
844/844 ━━━━━━━━━━━━━━━━━━━━ 7s 8ms/step - accuracy: 0.9916 - loss: 0.0244 - val_accuracy: 0.9903 - val_loss: 0.0311


In [9]:
score = cnn.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:',score[1])

Test Loss: 0.03027673251926899
Test accuracy: 0.9894000291824341


# Creating GUI

In [10]:
from keras.models import load_model
import customtkinter
import win32gui
from PIL import ImageGrab, Image
import numpy as np

In [11]:
# Load the pre-trained model (replace 'model.h5' with your actual path)
model = load_model('model.h5')

def predict_digit(img, top_n=3):
    """Preprocesses and predicts the digit from the image."""
    img = img.resize((28, 28))
    img = img.convert('L')
    img = np.array(img)
    img = img.reshape(1, 28, 28, 1)
    img = img/255.0

    res = model.predict([img])[0]
    sorted_indices = np.argsort(res)[-top_n:][::-1]
    top_predictions = [(i, res[i]) for i in sorted_indices]
    return top_predictions

class App(customtkinter.CTk):  # Inherit from CTk for custom styling
    """CustomTkinter-based app for digit recognition."""

    def __init__(self):
        super().__init__()
        self.title('Digit Recognizer')
        self.x = self.y = 0

        # Create custom CTk widgets for a visually appealing interface
        self.canvas = customtkinter.CTkCanvas(self, width=200, height=200, bg="black", cursor="cross")
        self.label = customtkinter.CTkLabel(self, text="Analyzing..", fg_color="gray", font=("Arial", 48))
        self.classify_btn = customtkinter.CTkButton(self, text="Predict", command=self.classify_handwriting)
        self.button_clear = customtkinter.CTkButton(self, text="Clear", command=self.clear_all)

        # Grid structure with customized padding and styling
        self.canvas.grid(row=0, column=0, rowspan=2, pady=20, padx=20, sticky="nsew")
        self.label.grid(row=0, column=1, rowspan=2, pady=20, padx=20, sticky="nsew")
        self.classify_btn.grid(row=2, column=0, columnspan=2, pady=10, padx=20, sticky="nsew")
        self.button_clear.grid(row=3, column=0, columnspan=2, pady=10, padx=20, sticky="nsew")

        # Bind events for drawing and classification
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        Hd = self.canvas.winfo_id()  # Handle of the canvas
        rect = win32gui.GetWindowRect(Hd)  # Get canvas edges

        try:
            im = ImageGrab.grab(rect)  # Capture the drawing area (replace if needed)
            top_predictions = predict_digit(im)

            prediction_text = '\n'.join([f"{digit}: {int(acc * 100)}%" for digit, acc in top_predictions])
            self.label.configure(text=prediction_text)

        except Exception as e:  # Handle potential errors (e.g., no drawing on canvas)
            print(f"Error during prediction: {e}")
            self.label.configure(text="Error: Please draw a digit on the canvas.")

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r = 10
        self.canvas.create_arc(self.x-r, self.y-r, self.x + r, self.y + r, fill='white')

# Run the application
app = App()
app.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
